# Car

In [38]:
import nest_asyncio
nest_asyncio.apply()

import asyncio, sys, os
sys.path.append(os.path.join(os.getenv('IOT49'), 'projects/robot/code-pi'))

from robot import RobotComm, PID_CPT1, PID_CPT2
from robot import LivePlot
from robot import Remote
import stm32, bokeh, time


class RemoteController(Remote):
    
    def __init__(self, rc: RobotComm):
        self.rc = rc
        self.speed = 0
        self.direction = 0
        super().__init__()
        
    def set_speed(self):
        duty1 = 100 * (self.speed+self.direction)
        duty2 = 100 * (self.speed-self.direction)
        print(f"set duty {int(duty1)} {int(duty2)}")
        rc.set_fstate('duty1', duty1)
        rc.set_fstate('duty2', duty2)
    
    async def handle(self, dt: float, code: str, value: float):
        if code == 'y':
            self.speed = value
            self.set_speed()
        elif code == 'x':
            self.direction = value
            self.set_speed()
        elif code == 'v':
            print(f"vbatt = {value:5.3} V")


print("reset")
stm32.hard_reset()
if False:
    import time
    print("rsync")
    stm32.rsync(dry_run=False)
    print("reset again")
    stm32.hard_reset()


def exception_handler(loop, context):
    msg = context.get("exception", context["message"])
    print("***** asyncio:", context)
    print("***** msg:", msg)
    
asyncio.get_event_loop().set_exception_handler(exception_handler)


async with RobotComm() as rc:
    if True:
        # init plotter
        bokeh.io.output_notebook()
        lp = LivePlot('duty1', ['diff_enc1', 'diff_enc2'], rollover=200, title="Duty Cycle")
        lp.show(rc.get_state_queue())
    # connect to remote and handle it's commands
    remote = RemoteController(rc)
    controller = 'duty_control'
    print(f"Start controller '{controller}'")
    rc.start_controller(fs=100, name=controller)
    asyncio.run(remote.run(peripheral_name='mpy-uart'))
    print(f"Stop controller")
    rc.stop_controller()
    print("DONE!")

reset
MCU: init controller



Loading BokehJS ...

Start controller 'duty_control'
scanning for mpy-uart
connecting to mpy-uart ... connected
vbatt =  4.25 V
set duty -5 5
set duty -9 9
set duty -13 13
set duty -18 18
set duty -21 21
set duty -25 25
set duty -28 22
set duty -32 26
set duty -37 31
set duty -42 35
set duty -45 38
set duty -31 25
set duty -10 4
set duty -6 8
set duty 0 1
set duty 3 -1
set duty 6 -4
set duty 11 -10
set duty 15 -13
set duty 20 -18
set duty 24 -22
set duty 29 -27
set duty 32 -30
set duty 40 -38
set duty 46 -44
set duty 49 -47
set duty 55 -53
set duty 33 -31
set duty 15 -13
set duty 5 -3
set duty 1 0
set duty 4 4
set duty 8 7
set duty 12 12
set duty 17 16
set duty 21 20
set duty 25 24
set duty 28 28
set duty 32 32
set duty 36 35
set duty 40 39
set duty 45 45
set duty 49 48
set duty 52 51
set duty 48 47
set duty 37 36
set duty 26 25
set duty 15 14
set duty 4 3
set duty 0 0
set duty -7 -7
set duty -3 -3
set duty 2 1
set duty -1 5
set duty 2 8
set duty -2 14
set duty 2 19
set duty -8 31
set duty -2 37
set duty -

In [37]:
import numpy as np

def d(speed, direction):
    d1 = int(speed+direction)
    d2 = int(speed-direction)
    mx = max(abs(d1), abs(d2))
    if mx < 1: mx = 1
    print(f"{speed:8.3f}   {direction:8.3f}   {d1/mx:6}  {d2/mx:6}   {mx:8.3f}")

for x in np.linspace(-1, 1, 9):
    for y in np.linspace(-1, 1, 3):
        d(x, y)

  -1.000     -1.000     -1.0     0.0      2.000
  -1.000      0.000     -1.0    -1.0      1.000
  -1.000      1.000      0.0    -1.0      2.000
  -0.750     -1.000     -1.0     0.0      1.000
  -0.750      0.000      0.0     0.0      1.000
  -0.750      1.000      0.0    -1.0      1.000
  -0.500     -1.000     -1.0     0.0      1.000
  -0.500      0.000      0.0     0.0      1.000
  -0.500      1.000      0.0    -1.0      1.000
  -0.250     -1.000     -1.0     0.0      1.000
  -0.250      0.000      0.0     0.0      1.000
  -0.250      1.000      0.0    -1.0      1.000
   0.000     -1.000     -1.0     1.0      1.000
   0.000      0.000      0.0     0.0      1.000
   0.000      1.000      1.0    -1.0      1.000
   0.250     -1.000      0.0     1.0      1.000
   0.250      0.000      0.0     0.0      1.000
   0.250      1.000      1.0     0.0      1.000
   0.500     -1.000      0.0     1.0      1.000
   0.500      0.000      0.0     0.0      1.000
   0.500      1.000      1.0     0.0    